In [54]:
# ============================================================
# 셀 0-2: 실험 플래그 및 하이퍼파라미터 설정 (⭐ 자주 바뀜)
# ============================================================
# ✅ 조건 1: Residual은 ON으로 고정 (이미 확인했으니)
USE_RESIDUAL_FIXED = True  # 항상 True로 고정

# 각 변수별 탐색 플래그 (한 번에 하나만 True)
# 🔥 실험 순서:
# Step 0) 기본 구조 고정: K=25, BIN_X=4.0, BIN_Y=5.0, Residual=True ✅ 완료
# Step 1) TOPK 탐색 - p90/p95 중심으로 평가 ✅ 완료 (TOPK=1로 고정)
# Step 2) Residual용 피처 추가 🔄 진행 중
# Step 3) CatBoost/앙상블 (예정)
K_SEARCH = False         # Step A: K만 탐색 (완료)
BIN_X_SEARCH = False     # Step B: BIN_X만 탐색 (완료)
BIN_Y_SEARCH = False     # Step C: BIN_Y만 탐색 (완료)
TOPK_SEARCH = False      # ✅ Step 1: TOPK 탐색 완료, 이제 고정

# 기본값 (Step 0 고정값)
# ✅ Step A/B/C의 최선값으로 설정 완료
K_DEFAULT = 25           # Step A 최선값 (K=25가 베스트)
TOPK_DEFAULT = 1         # Step 1 완료, TOPK=1로 고정
BIN_X_DEFAULT = 4.0      # Step B 최선값 (BIN_X=4.0이 베스트)
BIN_Y_DEFAULT = 5.0      # Step C 최선값 (BIN_Y=5.0이 베스트)

# 탐색할 값들 (플래그가 True일 때 사용)
K_CANDIDATES = [15, 20, 25]
BIN_X_CANDIDATES = [4.0, 5.0, 6.0]
BIN_Y_CANDIDATES = [5.0, 6.0, 7.0]
TOPK_CANDIDATES = [1, 3, 5]  # Step 1: TOPK 탐색 (p90/p95 중심 평가)

# 실험할 변수 결정
if K_SEARCH:
    K_LIST = K_CANDIDATES
    BIN_X_LIST = [BIN_X_DEFAULT]
    BIN_Y_LIST = [BIN_Y_DEFAULT]
    TOPK_LIST = [TOPK_DEFAULT]
    search_var = "K"
elif BIN_X_SEARCH:
    K_LIST = [K_DEFAULT]
    BIN_X_LIST = BIN_X_CANDIDATES
    BIN_Y_LIST = [BIN_Y_DEFAULT]
    TOPK_LIST = [TOPK_DEFAULT]
    search_var = "BIN_X"
elif BIN_Y_SEARCH:
    K_LIST = [K_DEFAULT]
    BIN_X_LIST = [BIN_X_DEFAULT]
    BIN_Y_LIST = BIN_Y_CANDIDATES
    TOPK_LIST = [TOPK_DEFAULT]
    search_var = "BIN_Y"
elif TOPK_SEARCH:
    K_LIST = [K_DEFAULT]
    BIN_X_LIST = [BIN_X_DEFAULT]
    BIN_Y_LIST = [BIN_Y_DEFAULT]
    TOPK_LIST = TOPK_CANDIDATES
    search_var = "TOPK"
else:
    # 모든 플래그가 False면 기본값만 사용
    K_LIST = [K_DEFAULT]
    BIN_X_LIST = [BIN_X_DEFAULT]
    BIN_Y_LIST = [BIN_Y_DEFAULT]
    TOPK_LIST = [TOPK_DEFAULT]
    search_var = "None (기본값만)"

print("🔧 실험 설정 (Residual ON 고정, 레벨1만 탐색)")
print(f" - 탐색 변수: {search_var}")
print(f" - USE_RESIDUAL: {USE_RESIDUAL_FIXED} (고정)")
print(f" - K: {K_LIST}")
print(f" - TOPK: {TOPK_LIST}")
print(f" - BIN_X: {BIN_X_LIST}")
print(f" - BIN_Y: {BIN_Y_LIST}")
print(f" - 총 조합 수: {len(K_LIST) * len(TOPK_LIST) * len(BIN_X_LIST) * len(BIN_Y_LIST)}")


🔧 실험 설정 (Residual ON 고정, 레벨1만 탐색)
 - 탐색 변수: None (기본값만)
 - USE_RESIDUAL: True (고정)
 - K: [25]
 - TOPK: [1]
 - BIN_X: [4.0]
 - BIN_Y: [5.0]
 - 총 조합 수: 1


In [55]:
# ============================================================
# 셀 0-5: 실험 루프 (⭐ 자주 바뀜)
# ============================================================
from sklearn.metrics import mean_squared_error

def train_lgbm_multiclass(X, y, num_class):
    """LightGBM multiclass 학습 함수"""
    dtrain = lgb.Dataset(X, label=y)
    params = {
        "objective":"multiclass",
        "num_class":num_class,
        "metric":"multi_logloss",
        "learning_rate":0.05,
        "num_leaves":31,
        "verbosity":-1
    }
    return lgb.train(params, dtrain, num_boost_round=200)

def train_residual(X, y):
    """LightGBM residual 회귀 학습 함수"""
    dtrain = lgb.Dataset(X, label=y)
    params = {
        "objective":"regression",
        "metric":"rmse",
        "learning_rate":0.05,
        "num_leaves":31,
        "verbosity":-1
    }
    return lgb.train(params, dtrain, num_boost_round=100)

def proba_to_coord(p, bin_size, k):
    """확률 벡터를 좌표로 변환 (top-k soft average)"""
    idx = np.argsort(p)[-k:]
    w = p[idx] / (p[idx].sum() + 1e-9)
    return np.sum((idx * bin_size + bin_size/2) * w)

# 실험 결과 저장
results = []
best_rec = None
best_mean_dist = float('inf')  # mean_dist 기준으로 베스트 선택

print("\n" + "=" * 90)
print("🚀 실험 루프 시작 (Residual ON 고정)")
print("=" * 90)

for K_var in K_LIST:
    for TOPK in TOPK_LIST:
        for BIN_X_var in BIN_X_LIST:
            for BIN_Y in BIN_Y_LIST:
                print("\n" + "-" * 90)
                print(f"📊 실험: K={K_var}, TOPK={TOPK}, BIN_X={BIN_X_var}, BIN_Y={BIN_Y} (Residual={USE_RESIDUAL_FIXED})")
                print("-" * 90)
                
                # K 값에 따라 wide feature 생성
                print("  - Wide feature 생성 중...")
                X_base = create_wide_features(events, K_var)
                print("  ✓ Wide feature 생성 완료")
                
                # BIN 라벨 생성
                print("  - BIN 라벨 생성 중...")
                NX = int(np.ceil(105 / BIN_X_var))
                NY = int(np.ceil(68 / BIN_Y))
                
                train_last_local = train_last_gt.copy()
                train_last_local["target_x_bin"] = (
                    train_last_local["end_x"] / BIN_X_var
                ).astype(int).clip(0, NX - 1)
                
                train_last_local["target_y_bin"] = (
                    train_last_local["end_y"] / BIN_Y
                ).astype(int).clip(0, NY - 1)
                
                X_all = X_base.merge(
                    train_last_local[["game_episode","target_x_bin","target_y_bin"]],
                    on="game_episode", how="left"
                )
                print("  ✓ BIN 라벨 생성 완료 (NX={}, NY={})".format(NX, NY))
                
                train_mask = X_all["target_x_bin"].notna()
                X_train_local = X_all[train_mask].copy()
                X_test_local = X_all[~train_mask].copy()
                
                drop_cols = ["game_episode","target_x_bin","target_y_bin"]
                X_train_feat_local = X_train_local.drop(columns=drop_cols)
                X_test_feat_local = X_test_local.drop(columns=drop_cols)
                
                y_train_x_local = X_train_local["target_x_bin"].astype(int)
                y_train_y_local = X_train_local["target_y_bin"].astype(int)
                
                print("  - 데이터 분할 완료 (Train: {}, Test: {})".format(len(X_train_local), len(X_test_local)))
                
                # Step 2: BIN 모델 학습 (1차 - Residual 피처 추가 전)
                print("  - BIN 모델 학습 중 (1차)...")
                model_x_local = train_lgbm_multiclass(X_train_feat_local, y_train_x_local, NX)
                model_y_local = train_lgbm_multiclass(X_train_feat_local, y_train_y_local, NY)
                print("  ✓ BIN 모델 학습 완료 (1차)")
                
                # BIN 예측 확률 얻기 (Residual 피처 생성용)
                proba_x_train = model_x_local.predict(X_train_feat_local)
                proba_y_train = model_y_local.predict(X_train_feat_local)
                
                # Step 2: Residual 피처 추가
                print("  - Step 2: Residual 피처 추가 중...")
                start_x_cols = [c for c in X_train_feat_local.columns if c.startswith("start_x_")]
                start_y_cols = [c for c in X_train_feat_local.columns if c.startswith("start_y_")]
                
                X_train_feat_with_residual = add_residual_features(
                    X_train_feat_local, proba_x_train, proba_y_train,
                    y_train_x_local.values, y_train_y_local.values,
                    BIN_X_var, BIN_Y, start_x_cols, start_y_cols
                )
                
                # Test에도 Residual 피처 추가 (BIN 예측 확률 필요)
                proba_x_test = model_x_local.predict(X_test_feat_local)
                proba_y_test = model_y_local.predict(X_test_feat_local)
                # Test는 실제 BIN이 없으므로 예측된 BIN 사용
                pred_bin_x_test = np.argmax(proba_x_test, axis=1)
                pred_bin_y_test = np.argmax(proba_y_test, axis=1)
                
                X_test_feat_with_residual = add_residual_features(
                    X_test_feat_local, proba_x_test, proba_y_test,
                    pred_bin_x_test, pred_bin_y_test,
                    BIN_X_var, BIN_Y, start_x_cols, start_y_cols
                )
                print("  ✓ Residual 피처 추가 완료")
                
                # Step 2: BIN 모델 재학습 (Residual 피처 포함)
                print("  - BIN 모델 재학습 중 (Residual 피처 포함)...")
                model_x_local = train_lgbm_multiclass(X_train_feat_with_residual, y_train_x_local, NX)
                model_y_local = train_lgbm_multiclass(X_train_feat_with_residual, y_train_y_local, NY)
                print("  ✓ BIN 모델 재학습 완료")
                
                # BIN 예측 (최종)
                proba_x_train = model_x_local.predict(X_train_feat_with_residual)
                proba_y_train = model_y_local.predict(X_train_feat_with_residual)
                
                pred_x_bin_train = np.array([proba_to_coord(p, BIN_X_var, TOPK) for p in proba_x_train])
                pred_y_bin_train = np.array([proba_to_coord(p, BIN_Y, TOPK) for p in proba_y_train])
                
                # Residual 처리 (항상 ON)
                if USE_RESIDUAL_FIXED:
                    print("  - Residual 모델 학습 중 (Residual 피처 포함)...")
                    residual_df = X_train_local[["game_episode"]].merge(
                        train_last_local, on="game_episode", how="left"
                    )
                    
                    bin_center_x_train = y_train_x_local.values * BIN_X_var + BIN_X_var/2
                    bin_center_y_train = y_train_y_local.values * BIN_Y + BIN_Y/2
                    
                    delta_x_train = residual_df["end_x"].values - bin_center_x_train
                    delta_y_train = residual_df["end_y"].values - bin_center_y_train
                    
                    # 클리핑
                    delta_x_train = np.clip(delta_x_train, -BIN_X_var/2, BIN_X_var/2)
                    delta_y_train = np.clip(delta_y_train, -BIN_Y/2, BIN_Y/2)
                    
                    # Step 2: Residual 피처가 포함된 피처로 학습
                    model_rx_local = train_residual(X_train_feat_with_residual, delta_x_train)
                    model_ry_local = train_residual(X_train_feat_with_residual, delta_y_train)
                    print("  ✓ Residual 모델 학습 완료")
                    
                    # Train 예측 (평가용) - Step 2: Residual 피처가 포함된 피처 사용
                    dx_pred_train = model_rx_local.predict(X_train_feat_with_residual)
                    dy_pred_train = model_ry_local.predict(X_train_feat_with_residual)
                    
                    pred_x_train = pred_x_bin_train + dx_pred_train
                    pred_y_train = pred_y_bin_train + dy_pred_train
                else:
                    # Residual 없이 BIN만 사용 (이론적으로는 발생 안 함)
                    pred_x_train = pred_x_bin_train
                    pred_y_train = pred_y_bin_train
                    model_rx_local = None
                    model_ry_local = None
                
                # Train RMSE 계산 (평가 기준)
                rmse_x = np.sqrt(mean_squared_error(train_last_local["end_x"].values, pred_x_train))
                rmse_y = np.sqrt(mean_squared_error(train_last_local["end_y"].values, pred_y_train))
                rmse_total = np.sqrt(rmse_x**2 + rmse_y**2)
                
                # mean_dist 계산 (직관적인 지표)
                dist = np.sqrt((pred_x_train - train_last_local["end_x"].values)**2 + 
                              (pred_y_train - train_last_local["end_y"].values)**2)
                mean_dist = np.mean(dist)
                p90 = np.percentile(dist, 90)
                p95 = np.percentile(dist, 95)
                pred_std_x = np.std(pred_x_train)
                pred_std_y = np.std(pred_y_train)
                
                print(f"  ✓ Train RMSE: X={rmse_x:.4f}, Y={rmse_y:.4f}, Total={rmse_total:.4f}")
                print(f"  ✓ Train mean_dist: {mean_dist:.4f}, p90: {p90:.4f}, p95: {p95:.4f}")
                print(f"  ✓ Train pred_std: X={pred_std_x:.4f}, Y={pred_std_y:.4f}")
                
                # 결과 저장
                rec = {
                    "K": K_var,
                    "USE_RESIDUAL": USE_RESIDUAL_FIXED,
                    "TOPK": TOPK,
                    "BIN_X": BIN_X_var,
                    "BIN_Y": BIN_Y,
                    "NX": NX,
                    "NY": NY,
                    "rmse_x": rmse_x,
                    "rmse_y": rmse_y,
                    "rmse_total": rmse_total,
                    "mean_dist": mean_dist,
                    "p90": p90,
                    "p95": p95,
                    "pred_std_x": pred_std_x,
                    "pred_std_y": pred_std_y,
                    "model_x": model_x_local,
                    "model_y": model_y_local,
                    "model_rx": model_rx_local if USE_RESIDUAL_FIXED else None,
                    "model_ry": model_ry_local if USE_RESIDUAL_FIXED else None,
                    "X_train_feat": X_train_feat_with_residual,
                    "X_test_feat": X_test_feat_with_residual,
                    "X_test": X_test_local,
                    "train_last_gt": train_last_local
                }
                results.append(rec)
                
                # 베스트 업데이트 (mean_dist가 가장 낮은 것 - 직관적인 지표)
                if mean_dist < best_mean_dist:
                    best_mean_dist = mean_dist
                    best_rec = rec
                    print(f"  🏆 새로운 베스트! mean_dist={mean_dist:.4f} (rmse_total={rmse_total:.4f})")
                
                # Step 2 완료 보고 (마지막 실험에서만)
                if K_var == K_LIST[-1] and TOPK == TOPK_LIST[-1] and BIN_X_var == BIN_X_LIST[-1] and BIN_Y == BIN_Y_LIST[-1]:
                    print("\n" + "=" * 90)
                    print("📊 Step 2 완료 보고")
                    print("=" * 90)
                    print("✅ 추가된 Residual 피처:")
                    residual_feat_cols = [c for c in X_train_feat_with_residual.columns 
                                         if c.startswith("bin_") or c.startswith("dist_to_") or 
                                         c.startswith("dx_from_") or c.startswith("dy_from_")]
                    print(f"   - 총 {len(residual_feat_cols)}개 피처 추가")
                    for feat in sorted(residual_feat_cols):
                        print(f"     • {feat}")
                    print(f"\n📈 Step 1 대비 성능 변화:")
                    print(f"   - mean_dist: {mean_dist:.4f}")
                    print(f"   - p90: {p90:.4f}")
                    print(f"   - p95: {p95:.4f}")
                    print(f"   - pred_std_x: {pred_std_x:.4f}, pred_std_y: {pred_std_y:.4f}")
                    print("=" * 90)



🚀 실험 루프 시작 (Residual ON 고정)

------------------------------------------------------------------------------------------
📊 실험: K=25, TOPK=1, BIN_X=4.0, BIN_Y=5.0 (Residual=True)
------------------------------------------------------------------------------------------
  - Wide feature 생성 중...


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_30600/2195510103.py:220: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos)


  ✓ Wide feature 생성 완료
  - BIN 라벨 생성 중...
  ✓ BIN 라벨 생성 완료 (NX=27, NY=14)
  - 데이터 분할 완료 (Train: 15435, Test: 2414)
  - BIN 모델 학습 중 (1차)...
  ✓ BIN 모델 학습 완료 (1차)
  - Step 2: Residual 피처 추가 중...
  ✓ Residual 피처 추가 완료
  - BIN 모델 재학습 중 (Residual 피처 포함)...
  ✓ BIN 모델 재학습 완료
  - Residual 모델 학습 중 (Residual 피처 포함)...
  ✓ Residual 모델 학습 완료
  ✓ Train RMSE: X=1.0252, Y=1.2091, Total=1.5853
  ✓ Train mean_dist: 1.4603, p90: 2.2682, p95: 2.4674
  ✓ Train pred_std: X=23.8324, Y=24.2954
  🏆 새로운 베스트! mean_dist=1.4603 (rmse_total=1.5853)

📊 Step 2 완료 보고
✅ 추가된 Residual 피처:
   - 총 37개 피처 추가
     • bin_center_x
     • bin_center_y
     • bin_entropy_x
     • bin_entropy_y
     • bin_top1_minus_top2_x
     • bin_top1_minus_top2_y
     • bin_top1_proba_x
     • bin_top1_proba_y
     • dist_to_goal_center
     • dist_to_sideline_0
     • dist_to_sideline_1
     • dist_to_sideline_10
     • dist_to_sideline_11
     • dist_to_sideline_12
     • dist_to_sideline_13
     • dist_to_sideline_14
     • dist_to_side

In [56]:
# ============================================================
# 셀 0-6: 베스트 설정 출력 및 제출 파일 생성 (거의 안 바뀜)
# ============================================================
# 필요한 함수 정의 (셀 0-5에서 정의된 함수 재정의)
def proba_to_coord(p, bin_size, k):
    """확률 벡터를 좌표로 변환 (top-k soft average)"""
    idx = np.argsort(p)[-k:]
    w = p[idx] / (p[idx].sum() + 1e-9)
    return np.sum((idx * bin_size + bin_size/2) * w)

# 10) 베스트 설정 출력
print("\n" + "=" * 90)
print("🏆 베스트 설정 (mean_dist 기준)")
print("=" * 90)
if best_rec is not None:
    print(f"K: {best_rec['K']}")
    print(f"USE_RESIDUAL: {best_rec['USE_RESIDUAL']} (고정)")
    print(f"TOPK: {best_rec['TOPK']}")
    print(f"BIN_X: {best_rec['BIN_X']}")
    print(f"BIN_Y: {best_rec['BIN_Y']}")
    print(f"mean_dist: {best_rec['mean_dist']:.4f} (선택 기준)")
    print(f"RMSE: X={best_rec['rmse_x']:.4f}, Y={best_rec['rmse_y']:.4f}, Total={best_rec['rmse_total']:.4f}")
else:
    print("⚠️ 베스트 결과가 없습니다. 셀 0-5를 먼저 실행하세요.")
print("=" * 90)

# 11) 베스트 설정으로 최종 예측 및 제출 파일 생성
print("\n" + "=" * 90)
print("📝 베스트 설정으로 최종 제출 파일 생성")
print("=" * 90)

if best_rec is None:
    print("⚠️ 베스트 결과가 없습니다. 셀 0-5를 먼저 실행하세요.")
else:

    # 베스트 모델 사용
    model_x_best = best_rec["model_x"]
    model_y_best = best_rec["model_y"]
    X_test_feat_best = best_rec["X_test_feat"]
    X_test_best = best_rec["X_test"]
    K_best = best_rec["K"]
    BIN_X_best = best_rec["BIN_X"]
    BIN_Y_best = best_rec["BIN_Y"]
    TOPK_best = best_rec["TOPK"]
    USE_RESIDUAL_best = best_rec["USE_RESIDUAL"]

    # BIN 예측 (Step 2: Residual 피처가 이미 포함된 피처 사용)
    proba_x_test = model_x_best.predict(X_test_feat_best)
    proba_y_test = model_y_best.predict(X_test_feat_best)

    pred_x_bin_test = np.array([proba_to_coord(p, BIN_X_best, TOPK_best) for p in proba_x_test])
    pred_y_bin_test = np.array([proba_to_coord(p, BIN_Y_best, TOPK_best) for p in proba_y_test])

    # Residual 적용 (Step 2: Residual 피처가 이미 포함된 피처 사용)
    if USE_RESIDUAL_best:
        dx_pred_test = best_rec["model_rx"].predict(X_test_feat_best)
        dy_pred_test = best_rec["model_ry"].predict(X_test_feat_best)
        
        pred_x_final = np.clip(pred_x_bin_test + dx_pred_test, 0, 105)
        pred_y_final = np.clip(pred_y_bin_test + dy_pred_test, 0, 68)
    else:
        pred_x_final = np.clip(pred_x_bin_test, 0, 105)
        pred_y_final = np.clip(pred_y_bin_test, 0, 68)

    # 제출 파일 생성
    pred_df = X_test_best[["game_episode"]].copy()
    pred_df["end_x"] = pred_x_final
    pred_df["end_y"] = pred_y_final

    sub = sample_sub.set_index("game_episode")
    pred_df = pred_df.set_index("game_episode")

    sub["end_x"] = pred_df["end_x"]
    sub["end_y"] = pred_df["end_y"]

    sub = sub.reset_index()
    sub.to_csv(FINAL_SUBMISSION_NAME, index=False)

    print(f"\n✅ 최종 제출 파일 생성 완료: {FINAL_SUBMISSION_NAME}")
    print(f"   베스트 설정: K={K_best}, USE_RESIDUAL={USE_RESIDUAL_best}, TOPK={TOPK_best}, BIN_X={BIN_X_best}, BIN_Y={BIN_Y_best}")
    print(f"   Train RMSE: {best_rec['rmse_total']:.4f}")
    print(f"\n제출 파일 경로: {os.path.abspath(FINAL_SUBMISSION_NAME)}")
    print("\n제출 파일 샘플:")
    print(sub.head())
    print(f"\n제출 파일 행 수: {len(sub)}")

    # 12) 실험 결과 요약
    print("\n" + "=" * 90)
    print("📊 전체 실험 결과 요약")
    print("=" * 90)
    if len(results) > 0:
        results_df = pd.DataFrame([
            {
                "K": r["K"],
                "USE_RESIDUAL": r["USE_RESIDUAL"],
                "TOPK": r["TOPK"],
                "BIN_X": r["BIN_X"],
                "BIN_Y": r["BIN_Y"],
                "RMSE_X": r["rmse_x"],
                "RMSE_Y": r["rmse_y"],
                "RMSE_Total": r["rmse_total"],
                "mean_dist": r["mean_dist"]
            }
            for r in results
        ])
        results_df = results_df.sort_values("RMSE_Total")
        print(results_df.to_string(index=False))
    else:
        print("⚠️ 실험 결과가 없습니다. 셀 0-5를 먼저 실행하세요.")



🏆 베스트 설정 (mean_dist 기준)
K: 25
USE_RESIDUAL: True (고정)
TOPK: 1
BIN_X: 4.0
BIN_Y: 5.0
mean_dist: 1.4603 (선택 기준)
RMSE: X=1.0252, Y=1.2091, Total=1.5853

📝 베스트 설정으로 최종 제출 파일 생성

✅ 최종 제출 파일 생성 완료: submission_lgbm_step2.csv
   베스트 설정: K=25, USE_RESIDUAL=True, TOPK=1, BIN_X=4.0, BIN_Y=5.0
   Train RMSE: 1.5853

제출 파일 경로: /Users/yangjinmo/Desktop/k_league_ml/submission_lgbm_step2.csv

제출 파일 샘플:
  game_episode      end_x      end_y
0     153363_1  61.834960   2.390788
1     153363_2  30.012772  67.630720
2     153363_6  54.005750  67.605873
3     153363_7  41.780530   1.286330
4     153363_8  78.340520   2.683488

제출 파일 행 수: 2414

📊 전체 실험 결과 요약
 K  USE_RESIDUAL  TOPK  BIN_X  BIN_Y   RMSE_X   RMSE_Y  RMSE_Total  mean_dist
25          True     1    4.0    5.0 1.025208 1.209133    1.585262   1.460279


In [57]:
# ============================================================
# ⚠️ 이 셀은 비활성화되었습니다. 셀 11 (셀 0-5)을 사용하세요.
# ============================================================
# 셀 11 (셀 0-5)이 이미 Step 2 피처 추가 코드를 포함하고 있으므로,
# 이 셀은 중복입니다.
# 
# 실행 순서 (처음부터 순서대로 실행하면 Step 2가 자동으로 반영됩니다):
# 1. 셀 0-1: Import 및 기본 설정
# 2. 셀 0-2: 실험 플래그 및 하이퍼파라미터 설정
# 3. 셀 0-3: 데이터 로드 및 전처리
# 4. 셀 0-4: Feature 생성 함수 (add_residual_features 포함)
# 5. 셀 11 (셀 0-5): 실험 루프 (Step 2 피처 추가 포함) ⭐ 이 셀을 실행하세요
# 6. 셀 12 (셀 0-6): 베스트 설정 출력 및 제출 파일 생성
# ============================================================
# 이 셀은 실행하지 마세요. 셀 11 (셀 0-5)을 실행하세요.
pass


In [58]:
# =======================
# 6️⃣ 추천 성능 체크 표
# (BIN Classification + Residual Regression Hybrid 버전)
# 실험마다 하나의 성능 체크 표 생성
# =======================

import pandas as pd
import numpy as np
import lightgbm as lgb

# =======================
# 실행 플래그
# =======================
RUN_PERFORMANCE_TABLE = True   # ✅ 성능 체크 표 생성
RUN_DETAILED_ANALYSIS = False  # 상세 분석 (필요시만)

# =======================
# 성능 체크 표 생성
# =======================
if RUN_PERFORMANCE_TABLE:
    print("=" * 90)
    print("6️⃣ 추천 성능 체크 표")
    print("=" * 90)
    
    try:
        if 'results' not in globals() or len(results) == 0:
            print("⚠️ 셀 0의 실험 결과(results)를 찾을 수 없습니다.")
            print("   셀 0을 먼저 실행하세요.")
        else:
            # 각 실험에 대한 성능 지표 계산
            performance_data = []
            
            def proba_to_coord(p, bin_size, k):
                """확률 벡터를 좌표로 변환 (top-k soft average)"""
                idx = np.argsort(p)[-k:]
                w = p[idx] / (p[idx].sum() + 1e-9)
                return np.sum((idx * bin_size + bin_size/2) * w)
            
            # 기본값 확인 (셀 0에서 가져오기)
            try:
                K_DEFAULT = K_DEFAULT if 'K_DEFAULT' in globals() else 20
                TOPK_DEFAULT = TOPK_DEFAULT if 'TOPK_DEFAULT' in globals() else 3
                BIN_X_DEFAULT = BIN_X_DEFAULT if 'BIN_X_DEFAULT' in globals() else 5.0
                BIN_Y_DEFAULT = BIN_Y_DEFAULT if 'BIN_Y_DEFAULT' in globals() else 6.0
            except:
                K_DEFAULT = 20
                TOPK_DEFAULT = 3
                BIN_X_DEFAULT = 5.0
                BIN_Y_DEFAULT = 6.0
            
            for rec in results:
                # 실험 이름 생성 (Residual은 항상 ON이므로 표시 안 함)
                # 🔥 현재 탐색 중인 Step에 맞게 해당 변수만 표시
                exp_name_parts = []
                
                # 현재 어떤 Step을 탐색 중인지 확인 (셀 0에서 가져오기)
                try:
                    K_SEARCH = K_SEARCH if 'K_SEARCH' in globals() else False
                    BIN_X_SEARCH = BIN_X_SEARCH if 'BIN_X_SEARCH' in globals() else False
                    BIN_Y_SEARCH = BIN_Y_SEARCH if 'BIN_Y_SEARCH' in globals() else False
                except:
                    K_SEARCH = False
                    BIN_X_SEARCH = False
                    BIN_Y_SEARCH = False
                
                # 현재 탐색 중인 변수만 표시
                if K_SEARCH and rec["K"] != K_DEFAULT:
                    exp_name_parts.append(f"K={rec['K']}")
                elif BIN_X_SEARCH and rec["BIN_X"] != BIN_X_DEFAULT:
                    exp_name_parts.append(f"BIN_X={rec['BIN_X']}")
                elif BIN_Y_SEARCH and rec["BIN_Y"] != BIN_Y_DEFAULT:
                    exp_name_parts.append(f"BIN_Y={rec['BIN_Y']}")
                
                # TOPK는 기본값(3) 고정이므로 표시 안 함
                # 모든 값이 기본값이면 "baseline" 표시
                exp_name = " | ".join(exp_name_parts) if exp_name_parts else "baseline"
                
                # Train 데이터에 대한 예측값 계산
                model_x = rec["model_x"]
                model_y = rec["model_y"]
                X_train_feat = rec["X_train_feat"]
                train_last_gt = rec["train_last_gt"]
                
                # BIN 예측
                proba_x = model_x.predict(X_train_feat)
                proba_y = model_y.predict(X_train_feat)
                
                pred_x_bin = np.array([proba_to_coord(p, rec["BIN_X"], rec["TOPK"]) for p in proba_x])
                pred_y_bin = np.array([proba_to_coord(p, rec["BIN_Y"], rec["TOPK"]) for p in proba_y])
                
                # Residual 적용
                if rec["USE_RESIDUAL"] and rec["model_rx"] is not None:
                    dx_pred = rec["model_rx"].predict(X_train_feat)
                    dy_pred = rec["model_ry"].predict(X_train_feat)
                    pred_x = pred_x_bin + dx_pred
                    pred_y = pred_y_bin + dy_pred
                else:
                    pred_x = pred_x_bin
                    pred_y = pred_y_bin
                
                # 실제값
                true_x = train_last_gt["end_x"].values
                true_y = train_last_gt["end_y"].values
                
                # 거리 계산 (유클리드 거리)
                dist = np.sqrt((pred_x - true_x)**2 + (pred_y - true_y)**2)
                
                # 지표 계산
                mean_dist = np.mean(dist)
                p90 = np.percentile(dist, 90)
                p95 = np.percentile(dist, 95)
                pred_std_x = np.std(pred_x)
                pred_std_y = np.std(pred_y)
                
                performance_data.append({
                    "실험": exp_name,
                    "mean_dist": mean_dist,
                    "p90": p90,
                    "p95": p95,
                    "pred_std_x": pred_std_x,
                    "pred_std_y": pred_std_y
                })
            
            # 표 생성
            perf_df = pd.DataFrame(performance_data)
            
            # Step 1 (TOPK 탐색)에서는 p90 중심으로 정렬, 그 외에는 mean_dist 기준
            try:
                TOPK_SEARCH = TOPK_SEARCH if 'TOPK_SEARCH' in globals() else False
            except:
                TOPK_SEARCH = False
            
            if TOPK_SEARCH:
                # Step 1: p90 중심 정렬 (큰 실수 감소가 목표)
                perf_df = perf_df.sort_values("p90")
                sort_criterion = "p90"
                print("\n📊 성능 체크 표 (p90 기준 정렬 - Step 1: TOPK 탐색)")
            else:
                # 기존: mean_dist 기준 정렬
                perf_df = perf_df.sort_values("mean_dist")
                sort_criterion = "mean_dist"
                print("\n📊 성능 체크 표 (mean_dist 기준 정렬)")
            
            print("=" * 90)
            display_df = perf_df[["실험", "mean_dist", "p90", "p95", "pred_std_x", "pred_std_y"]].copy()
            display_df["mean_dist"] = display_df["mean_dist"].round(4)
            display_df["p90"] = display_df["p90"].round(4)
            display_df["p95"] = display_df["p95"].round(4)
            display_df["pred_std_x"] = display_df["pred_std_x"].round(4)
            display_df["pred_std_y"] = display_df["pred_std_y"].round(4)
            print(display_df.to_string(index=False))
            
            if TOPK_SEARCH:
                print("\n💡 Step 1: TOPK 탐색 - 큰 실수 감소가 목표")
                print("   ⭐ p90, p95: 낮을수록 좋음 (큰 실수 감소) - 주요 평가 지표")
                print("   - mean_dist: 보조 지표 (평균 예측 오차)")
                print("   - pred_std_x, pred_std_y: 적절한 범위여야 함 (너무 작으면 다양성 부족)")
            else:
                print("\n💡 참고: 숫자보다 방향성을 봅니다.")
                print("   - mean_dist: 낮을수록 좋음 (평균 예측 오차)")
                print("   - p90, p95: 낮을수록 좋음 (예측의 안정성)")
                print("   - pred_std_x, pred_std_y: 적절한 범위여야 함 (너무 작으면 다양성 부족)")
            
            # 베스트 표시 (mean_dist 기준 - 셀 0에서 선택한 것)
            if 'best_rec' in globals():
                best_idx = next((i for i, r in enumerate(results) if r == best_rec), None)
                if best_idx is not None:
                    best_row = perf_df.iloc[best_idx]
                    print(f"\n🏆 베스트 실험 (mean_dist 기준): {best_row['실험']}")
                    print(f"   - mean_dist: {best_row['mean_dist']:.4f} (선택 기준)")
                    print(f"   - rmse_total: {best_rec['rmse_total']:.4f}")
                    print(f"   - p90: {best_row['p90']:.4f}, p95: {best_row['p95']:.4f}")
                    
                    # 베스트 설정 상세 정보
                    print(f"\n📌 베스트 설정 상세:")
                    print(f"   - K: {best_rec['K']}")
                    print(f"   - BIN_X: {best_rec['BIN_X']}")
                    print(f"   - BIN_Y: {best_rec['BIN_Y']}")
                    print(f"   - TOPK: {best_rec['TOPK']} (고정)")
                    print(f"   - USE_RESIDUAL: {best_rec['USE_RESIDUAL']} (고정)")
                    print(f"   - mean_dist: {best_rec['mean_dist']:.4f} (선택 기준)")
                    print(f"   - Train RMSE: X={best_rec['rmse_x']:.4f}, Y={best_rec['rmse_y']:.4f}, Total={best_rec['rmse_total']:.4f}")
            
    except Exception as e:
        print(f"⚠️ 성능 체크 표 생성 중 오류: {e}")
        import traceback
        traceback.print_exc()

# =======================
# 상세 분석 (선택사항)
# =======================
if RUN_DETAILED_ANALYSIS:
    print("\n" + "=" * 90)
    print("📊 상세 분석")
    print("=" * 90)
    
    try:
        if 'best_rec' in globals():
            print(f"\n베스트 설정:")
            print(f"  - USE_RESIDUAL: {best_rec['USE_RESIDUAL']}")
            print(f"  - TOPK: {best_rec['TOPK']}")
            print(f"  - BIN_X: {best_rec['BIN_X']}")
            print(f"  - BIN_Y: {best_rec['BIN_Y']}")
            print(f"  - Train RMSE: X={best_rec['rmse_x']:.4f}, Y={best_rec['rmse_y']:.4f}, Total={best_rec['rmse_total']:.4f}")
            
            # 제출 파일 정보
            try:
                submission_filename = FINAL_SUBMISSION_NAME
                sub = pd.read_csv(submission_filename)
                print(f"\n제출 파일: {submission_filename}")
                print(f"제출 파일 샘플:")
                print(sub.head(10))
            except:
                pass
    except:
        pass
else:
    print("\n상세 분석 건너뜀")




6️⃣ 추천 성능 체크 표

📊 성능 체크 표 (mean_dist 기준 정렬)
      실험  mean_dist    p90    p95  pred_std_x  pred_std_y
baseline     1.4603 2.2682 2.4674     23.8324     24.2954

💡 참고: 숫자보다 방향성을 봅니다.
   - mean_dist: 낮을수록 좋음 (평균 예측 오차)
   - p90, p95: 낮을수록 좋음 (예측의 안정성)
   - pred_std_x, pred_std_y: 적절한 범위여야 함 (너무 작으면 다양성 부족)

🏆 베스트 실험 (mean_dist 기준): baseline
   - mean_dist: 1.4603 (선택 기준)
   - rmse_total: 1.5853
   - p90: 2.2682, p95: 2.4674

📌 베스트 설정 상세:
   - K: 25
   - BIN_X: 4.0
   - BIN_Y: 5.0
   - TOPK: 1 (고정)
   - USE_RESIDUAL: True (고정)
   - mean_dist: 1.4603 (선택 기준)
   - Train RMSE: X=1.0252, Y=1.2091, Total=1.5853

상세 분석 건너뜀
